## Toy example: Inferring the mean of Gaussians

#### comparing the multi-round SNPE approach against our new incremental approach.

Goal of this little toy example is to show that provided our parameters are independent of each other, we need less simulations to derive a good approximation of our parameters.

In [1]:
import sys
sys.path.append('../code/')

import utils
from utils.helpers import get_time
from utils import inference

from utils.sbi_modulated_functions import Combined


from utils.helpers import get_time



# sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.inference import SNPE_C

import sbi

import pyknos

In [2]:
print(sbi.__version__)

0.17.2


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch

def Gaussian(thetas, normal_noise=1):
    
    gauss_list = []
    
    for theta in thetas:
    
        mu, sigma = theta, normal_noise # mean and standard deviation

        s = np.random.normal(mu, sigma, 1)
        
        gauss_list.append(s[0])
        
    gauss_obs = torch.tensor(gauss_list)
    
    return gauss_obs
    



### Calculate posterior for different number of simulations: 1k,  3k, 5k, 10k

### starting with multi-round snpe

In [4]:
true_thetas = torch.tensor([[3.0, 6.0, 20.0, 10.0, 90.0, 55.0, 27.0, 29.0, 4.0, 70.0, 5.0, 66.0, 99.0, 40.0, 45.0]], dtype=torch.float32)
parameter_names = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15']

prior_max = [100.0] * 15
prior_min = [0.1] * 15


import datetime

In [5]:
print(true_thetas)

tensor([[ 3.,  6., 20., 10., 90., 55., 27., 29.,  4., 70.,  5., 66., 99., 40.,
         45.]])


In [8]:
num_simulations_list = [200, 500]

torch.manual_seed(5)
np.random.seed(5)


posterior_snpe_list = []

obs_real = Gaussian(true_thetas[0], 1)

#obs_real = torch.tensor(obs_real, dtype=torch.float32)


for num_simulations in num_simulations_list:
    
    
    prior = utils.torchutils.BoxUniform(low=prior_min, high = prior_max)
    inf = SNPE(prior, density_estimator="nsf")
    simulator_stats, prior = prepare_for_sbi(Gaussian, prior)

    proposal = prior
    

    for i in range(3):
        
        start_time = datetime.datetime.now()

        theta, x = simulate_for_sbi(
            simulator_stats,
            proposal=proposal,
            num_simulations=num_simulations,
            num_workers=8,
        )

        inf = inf.append_simulations(theta, x, proposal=proposal)
        density_estimator = inf.train()

        posterior = inf.build_posterior(density_estimator)



        proposal = posterior.set_default_x(obs_real)
        
        finish_time = datetime.datetime.now()

        diff_time_snpe = finish_time - start_time
        
        print('for round ',i, ' time is: ', diff_time_snpe)

    posterior_snpe = proposal
    
    posterior_snpe_list.append(posterior_snpe)
    
finish_time = datetime.datetime.now()

diff_time_snpe = finish_time - start_time

Running 200 simulations in 200 batches.:   0%|          | 0/200 [00:00<?, ?it/s]

Neural network successfully converged after 318 epochs.
for round  0  time is:  0:00:30.466418


Drawing 200 posterior samples:   0%|          | 0/200 [00:00<?, ?it/s]

Running 200 simulations in 200 batches.:   0%|          | 0/200 [00:00<?, ?it/s]

Using SNPE-C with atomic loss
Neural network successfully converged after 29 epochs.
for round  1  time is:  0:00:17.304356


...

Using SNPE-C with atomic loss
Neural network successfully converged after 68 epochs.
for round  1  time is:  0:01:45.652024


Drawing 500 posterior samples:   0%|          | 0/500 [00:00<?, ?it/s]

                        constant for `log_prob()`. However, only
                        0% posterior samples are within the
                        prior support. It may take a long time to collect the
                        remaining 499 samples.
                        Consider interrupting (Ctrl-C) and either basing the
                        estimate of the normalizing constant on fewer samples (by
                        calling `posterior.leakage_correction(x_o,
                        num_rejection_samples=N)`, where `N` is the number of
                        samples you want to base the
                        estimate on (default N=10000), or not estimating the
                        normalizing constant at all
                        (`log_prob(..., norm_posterior=False)`. The latter will
                        result in an unnormalized `log_prob()`.


Running 500 simulations in 500 batches.:   0%|          | 0/500 [00:00<?, ?it/s]

Using SNPE-C with atomic loss
Neural network successfully converged after 56 epochs.
for round  2  time is:  0:02:27.631997


In [7]:
print(diff_time_snpe)

0:00:04.751526
